## Trabalho Implementação PNL

##### William Lucas e Nayara Gomes

In [4]:
import math
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML, display

Função e domínio:

![Função](./funcao.png)

In [5]:
def funcao(x1, x2):
    return -1 * math.exp(-1*(x1**2 + x2**2))

##### Funções auxiliares

Para calcular o gradiente, utilizamos a definição abaixo:

![Gradiente](./gradiente.png)

In [6]:
def gradiente(x1, x2):
    grad = np.array([[x1 * 2 * math.exp(-1 * (x1**2 + x2**2))], 
                   [x2 * 2 * math.exp(-1 * (x1**2 + x2**2))]])
    return grad

Para calcular a Matriz Hessiana, utilizamos a definição abaixo:

![Hessiana](./hessiana.png)

In [7]:
def matriz_hessiana(x1, x2):
    hess = np.array([[(2 - 4 * x1**2) * math.exp(-1 * (x1**2 + x2**2)),  - 4 * x1 * x2 * math.exp(-1 * (x1**2 + x2**2))],
                  [-4 * x1 * x2 * math.exp(-1 * (x1**2 + x2**2)), (2 - 4 * x2 **2) * math.exp(-1 * (x1**2 + x2**2)) ]])
    return hess

##### 1. Comportamento da função

Contínua: Sim, pois não há valores para x1 e x2 fora do domínio.

Mínimo local: A função não tem mínimo local.

Mínimo global: f(0,0) = -1

Assíntota: 

Convexidade: 